<a href="https://colab.research.google.com/github/YinterestingProjects/human-wildlife-interactions/blob/main/notebooks/BERTopics_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install pandas==1.4.2 bertopic==0.14.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 82.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pickle
import pandas as pd
from bertopic import BERTopic
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer

RANDOM_SEED = 42

# directory setup
user = "sally"
#user = "jina"
#user = "josh"
     
try:
  if user == "sally":
    directory = '/content/drive/MyDrive/MADS/MADS_Capstone'
  elif user == "jina":
    directory = "JINA: INSERT YOUR PATH HERE"
  elif user == "josh":
    directory = "/content/drive/MyDrive/MADS_Capstone"
except:
  dprint("No user has been selected")

In [4]:
fp = f'{directory}/data/processed/desc_title_translated.pkl'
corpus = pd.read_pickle(fp)

In [6]:
def filter_null(df, verbose=True):
    '''filter null translations'''
    clean_corp = df[(df != ' ') & (df != '') & (df != 'source language unavailable for translation')]  
    if verbose:
        print(f'{len(df) - len(clean_corp)} null records dropped out of {len(df)}')  
    return clean_corp

In [10]:
fp = f'{directory}/data/processed/desc_title_translated.pkl'
corpus = pd.read_pickle(fp)
  
order = corpus.index
new_doc = filter_null(corpus['title_en']).reset_index()
docs = new_doc['title_en']
docs
new_doc['index']

26 null records dropped out of 3895


0          0
1          1
2          2
3          3
4          4
        ... 
3864    3890
3865    3891
3866    3892
3867    3893
3868    3894
Name: index, Length: 3869, dtype: int64

In [83]:
new_doc['index'][3868]

3894

In [ ]:
# ----------- topic modeling results --------------
# X [1]loop through search terms
# X [2]loop through similarity query threshold
# [3]loop through topics that are above threshold
# [4]find threshold cut offs of relevant videos from video-topics matix 
# [5]if video above threshold mark 1, else mark 0

# ----------- human annotation results --------------
# [1]loop through all available topics
# [2]find threshold cut offs of relevant videos from video-topics matix  (0.5) lower the better check n of video total to review
# [3][manual] loop through videos, show video, record human annonatation of hunting activity present (1) or not (0)


''''         Human Annot.
                0  1 
              -------
            0 |TN|FN|
 Topic        -------
 Modeling   1 |FP|TP|
              -------
''''

# threshold vs precision + recal curve
# precison = TP/(TP+FP)     
# recall = TP/(TP+FN)

In [ ]:
# loop through all available topics
# find threshold cutoffs of relevant videos from video-topics matix  (0.5) lower the better check n of video total to review

{
topic1:[(video index, 0.3),(video index, 0.6), (video index, 0.9)],
topic1:[(video index, 0.3),(video index, 0.6), (video index, 0.9)],
topic1:[(video index, 0.3),(video index, 0.6), (video index, 0.9)],
...
 }

# grab video per topic > 0.5 for human labeling




In [34]:
def get_topic_video_lookup(topics, probs):
  video_topic = list(zip(topics, probs))

  topic_video_lookup = {}
  for v_id, (topic, prob) in enumerate(video_topic):
    if topic not in topic_video_lookup:
      topic_video_lookup[topic] = [(v_id, prob)]
    else:
      topic_video_lookup[topic].append((v_id, prob))
  return topic_video_lookup

In [42]:
model_fp = f'{directory}/models/bertopic_trigram'
topic_model = BERTopic.load(model_fp)
topics, probs = topic_model.fit_transform(docs)

In [43]:
trigram_tp_lookup = get_topic_video_lookup(topics, probs)

In [39]:
bigram_tp_lookup = get_topic_video_lookup(topics, probs)

In [36]:
guided_tp_lookup = get_topic_video_lookup(topics, probs)

In [46]:
print(f'outlier videos count guided {len(guided_tp_lookup[-1])}')
print(f'outlier videos count bigram {len(bigram_tp_lookup[-1])}')
print(f'outlier videos count trigram {len(trigram_tp_lookup[-1])}')

outlier videos count guided 1126
outlier videos count bigram 1281
outlier videos count trigram 1281


In [77]:
# filter list by threshold 0.5, find total

def count_videos(lookup):
  count = 0
  for topic in lookup:
    count += len(lookup[topic]) 
  return count

def count_empty_topics(lookup):
  count = 0
  for topic in lookup:
    if len(lookup[topic]) == 0:
      count += 1
      #print(topic)
    return count

def get_videos_by_topic_threshold(lookup, threshold=0.5):
  '''filter a video topic lookup based on threshold of topic video '''
  
  filtered_videos = {}
  for topic in lookup:
    filtered_videos[topic] = [(v_id, prob) for v_id, prob in lookup[topic] if prob > threshold]
    
  print(f'videos remaining: {count_videos(filtered_videos)}/{count_videos(lookup)}, empty topic count {count_empty_topics(filtered_videos)}/{count_empty_topics(lookup)}')
  return filtered_videos

In [78]:
filtered = get_videos_by_topic_threshold(trigram_tp_lookup, threshold=0.5)
filtered = get_videos_by_topic_threshold(trigram_tp_lookup, threshold=0.6)
filtered = get_videos_by_topic_threshold(trigram_tp_lookup, threshold=0.7)
filtered = get_videos_by_topic_threshold(trigram_tp_lookup, threshold=0.9)
filtered = get_videos_by_topic_threshold(trigram_tp_lookup, threshold=0.95)
filtered = get_videos_by_topic_threshold(trigram_tp_lookup, threshold=0.999999)

videos remaining: 2330/3869, empty topic count 1/0
videos remaining: 2178/3869, empty topic count 1/0
videos remaining: 1995/3869, empty topic count 1/0
videos remaining: 1510/3869, empty topic count 1/0
videos remaining: 1339/3869, empty topic count 1/0
videos remaining: 1187/3869, empty topic count 1/0


In [79]:
filtered = get_videos_by_topic_threshold(bigram_tp_lookup, threshold=0.5)
filtered = get_videos_by_topic_threshold(bigram_tp_lookup, threshold=0.6)
filtered = get_videos_by_topic_threshold(bigram_tp_lookup, threshold=0.7)
filtered = get_videos_by_topic_threshold(bigram_tp_lookup, threshold=0.9)
filtered = get_videos_by_topic_threshold(bigram_tp_lookup, threshold=0.95)
filtered = get_videos_by_topic_threshold(bigram_tp_lookup, threshold=0.999999)

videos remaining: 2330/3869, empty topic count 1/0
videos remaining: 2178/3869, empty topic count 1/0
videos remaining: 1995/3869, empty topic count 1/0
videos remaining: 1510/3869, empty topic count 1/0
videos remaining: 1339/3869, empty topic count 1/0
videos remaining: 1187/3869, empty topic count 1/0


In [80]:
filtered = get_videos_by_topic_threshold(guided_tp_lookup, threshold=0.5)
filtered = get_videos_by_topic_threshold(guided_tp_lookup, threshold=0.6)
filtered = get_videos_by_topic_threshold(guided_tp_lookup, threshold=0.7)
filtered = get_videos_by_topic_threshold(guided_tp_lookup, threshold=0.9)
filtered = get_videos_by_topic_threshold(guided_tp_lookup, threshold=0.95)
filtered = get_videos_by_topic_threshold(guided_tp_lookup, threshold=0.999999)

videos remaining: 2316/3869, empty topic count 1/0
videos remaining: 2074/3869, empty topic count 1/0
videos remaining: 1850/3869, empty topic count 1/0
videos remaining: 1227/3869, empty topic count 1/0
videos remaining: 1052/3869, empty topic count 1/0
videos remaining: 858/3869, empty topic count 1/0


In [ ]:
# randomly sample 300? 

In [88]:
def update_video_index(filtered_lookup, index_key):
  '''updating video indexes of the filtered topic video look to original video dataframe index'''

  updated_lookup = {}
  for topic in filtered_lookup:
    video_list = []
    for v_id, prob in filtered_lookup[topic]:
      video_list.append((index_key[v_id],prob))
    updated_lookup[topic] = video_list
  
  return updated_lookup 

In [86]:
filtered[1][:10]

[(4, 1.0),
 (5, 1.0),
 (195, 1.0),
 (451, 1.0),
 (566, 1.0),
 (619, 1.0),
 (629, 1.0),
 (707, 1.0),
 (780, 1.0),
 (790, 1.0)]

In [89]:
updated_filtered = update_video_index(filtered, new_doc['index'])
updated_filtered[1][:10]

[(4, 1.0),
 (5, 1.0),
 (200, 1.0),
 (456, 1.0),
 (571, 1.0),
 (624, 1.0),
 (634, 1.0),
 (714, 1.0),
 (787, 1.0),
 (797, 1.0)]